# Dados da Estabelecimentos de Saúde (CNES) com data base de 12/2022 #####
Arquivo: Dados/tbEstabelecimento202212.csv

Fonte: https://cnes.datasus.gov.br/pages/downloads/arquivosBaseDados.jsp

Data-base: 12/2022

## Dados de UBS em Minas Gerais
Filtro: [TP_UNIDADE] = 01 ou 02 (Posto de Saúde ou Unidade de Saúde)

Filtro: [CO_ESTADO_GESTOR] = 31 (Estado de Minas Gerais)

Filtro: [CO_MOTIVO_DESAB] = nulo/vazio (Apenas estabelecimentos que estão ativos)

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

input_path = Path("Dados/tbEstabelecimento202212.csv")   # ajuste se necessário
output_path = Path("ubs_mg.xlsx")
output_sheet_name = "ubs_mg"

def fix_coord_one(value):
    """Normaliza lat/long em formato BR: sinal + 2 dígitos + vírgula + resto.
       Ex.: -20685 -> -20,685 | -21.12 -> -21,12 | -19,784,674 -> -19,784674
    """
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return np.nan
    s = str(value).strip()
    if s == "" or s.lower() in {"nan","null","none"}:
        return np.nan

    # captura sinal
    sign = "-" if "-" in s else "+"
    # extrai apenas dígitos
    digits = "".join(re.findall(r"\d", s))
    if digits == "":
        return np.nan

    # se tiver 1–2 dígitos, não há parte decimal → adiciona ,0
    if len(digits) <= 2:
        return f"{'-' if sign=='-' else ''}{digits},0"

    # monta: 2 dígitos + vírgula + restante
    deg = digits[:2]
    frac = digits[2:]
    return f"{'-' if sign=='-' else ''}{deg},{frac}"


def pick_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

# leia o CSV 
df = pd.read_csv(input_path, sep=";", dtype=str, encoding="latin-1")

# 1) TP_UNIDADE = 1/2 (também 01/02)
tp_col = pick_column(df, ["TP_UNIDADE"])
df["_TP_"] = df[tp_col].astype(str).str.strip().str.zfill(2)
df = df[df["_TP_"].isin({"01","02"})].copy()
df.drop(columns=["_TP_"], inplace=True)

# 2) Minas Gerais (CO_ESTADO_GESTOR = 31)
uf_col = pick_column(df, ["CO_ESTADO_GESTOR"])
df = df[df[uf_col].astype(str).str.strip() == "31"].copy()

# 3) Não desabilitado (CO_MOTIVO_DESAB nulo/vazio)
desab_col = pick_column(df, ["CO_MOTIVO_DESAB"])
if desab_col in df.columns:
    mask_null = df[desab_col].isna() | (df[desab_col].astype(str).str.strip().isin({"","NULL","null"}))
    df = df[mask_null].copy()

# 4) Corrigir latitude/longitude (com vírgula)
df["NU_LATITUDE"]  = df["NU_LATITUDE"].apply(fix_coord_one)
df["NU_LONGITUDE"] = df["NU_LONGITUDE"].apply(fix_coord_one)

# salva Excel
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name=output_sheet_name, index=False)

print("OK ->", output_path)


OK -> ubs_mg.xlsx


## Preparação dos dados por Municípios no estado de Minas Gerais
arquivo: MG_Municipios_2024.gpkg

Fonte: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html

Data-base: 2024

arquivos: Agregados_por_municipio_mortalidade_MG.xlsx
Fonte: https://datasus.saude.gov.br/mortalidade-desde-1996-pela-cid-10
Data-base: 2022

arquivos: Agregados_por_municipio_atlasivs_dadosbrutos_ipea.xlsx
Fonte: https://ivs.ipea.gov.br/#/
Data-base: 2010

arquivos: Agregador_por_municipio_pop_area_MG.xlsx
Fonte: https://www.ibge.gov.br/estatisticas/sociais/populacao/22827-censo-demografico-2022.html?edicao=37225&t=resultados
Data-base: 2022

Lista de CID da ICSAP: PORTARIA Nº 221, DE 17 DE ABRIL DE 2008

## Dados socioeconomicos por município

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

# =========================================================
# CONFIGURAÇÃO DE CAMINHOS
# =========================================================
base_dir = Path("Dados")

path_mort    = base_dir / "Agregados_por_municipio_mortalidade_MG.xlsx"
path_ivs     = base_dir / "Agregados_por_municipio_atlasivs_dadosbrutos_ipea.xlsx"
path_pop     = base_dir / "Agregador_por_municipio_pop_area_MG.xlsx"

path_saida   = Path("caract_municipio_mg.xlsx")

# =========================================================
# FUNÇÕES AUXILIARES
# =========================================================

def cria_ibge6(col):
    """
    Extrai apenas dígitos e monta o código IBGE6:
    - se houver >= 7 dígitos, pega os 6 primeiros (ex.: 3100104 -> 310010)
    - se houver 6 dígitos, usa como está
    - se houver < 6, faz zfill(6)
    """
    s = col.astype(str).str.replace(r"\D", "", regex=True)  # só dígitos

    def conv(x):
        if pd.isna(x) or x == "":
            return np.nan
        if len(x) >= 7:
            return x[:6]
        elif len(x) == 6:
            return x
        else:
            return x.zfill(6)

    return s.apply(conv)

def pick(df, candidates):
    """Escolhe a primeira coluna existente em uma lista de candidatos."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

# =========================================================
# 1) df_icsap  (aba 2022-ICSAP)
# =========================================================
df_icsap = pd.read_excel(path_mort, sheet_name="2022-ICSAP", dtype=str)

# coluna Município -> IBGE6
col_mun_icsap = pick(df_icsap, ["Município", "Municipio", "CD_MUN", "cod_mun"])
if col_mun_icsap is None:
    raise ValueError("Não encontrei coluna de município em 2022-ICSAP.")

df_icsap["IBGE6"] = cria_ibge6(df_icsap[col_mun_icsap])

# renomear coluna de óbitos p/ residência
col_obitos_icsap = pick(df_icsap, ["Óbitos p/Residênc", "Óbitos p/Residência", "Obitos p/Residenc"])
if col_obitos_icsap is None:
    raise ValueError("Não encontrei coluna 'Óbitos p/Residênc' em 2022-ICSAP.")
df_icsap = df_icsap[["IBGE6", col_obitos_icsap]].copy()
df_icsap = df_icsap.rename(columns={col_obitos_icsap: "obitos_icsap"})

# =========================================================
# 2) df_total  (aba 2022-Total)
# =========================================================
df_total = pd.read_excel(path_mort, sheet_name="2022-Total", dtype=str)

col_mun_total = pick(df_total, ["Município", "Municipio", "CD_MUN", "cod_mun"])
if col_mun_total is None:
    raise ValueError("Não encontrei coluna de município em 2022-Total.")

df_total["IBGE6"] = cria_ibge6(df_total[col_mun_total])

col_obitos_total = pick(df_total, ["Óbitos p/Residênc", "Óbitos p/Residência", "Obitos p/Residenc"])
if col_obitos_total is None:
    raise ValueError("Não encontrei coluna 'Óbitos p/Residênc' em 2022-Total.")
df_total = df_total[["IBGE6", col_obitos_total]].copy()
df_total = df_total.rename(columns={col_obitos_total: "obitos_total"})

# =========================================================
# 3) df_obitos = merge de df_total e df_icsap por IBGE6
# =========================================================
df_obitos = df_total.merge(df_icsap, on="IBGE6", how="left")
df_obitos = df_obitos[["IBGE6", "obitos_total", "obitos_icsap"]].copy()

# =========================================================
# 4) df_ivs  (Dados brutos IVS)
# =========================================================
df_ivs = pd.read_excel(
    path_ivs,
    sheet_name="Dados brutos IVS",
    dtype=str
)

# renomear municipio_6digt -> IBGE6
if "municipio_6digt" not in df_ivs.columns:
    raise ValueError("Não encontrei coluna 'municipio_6digt' em Dados brutos IVS.")
df_ivs = df_ivs.rename(columns={"municipio_6digt": "IBGE6"})

# filtros:
df_ivs = df_ivs[
    (df_ivs["nivel"] == "regiao,uf,rm,municipio") &
    (df_ivs["ano"] == "2010") &
    (df_ivs["uf"] == "31") &
    (df_ivs["label_cor"] == "Total Cor") &
    (df_ivs["label_sexo"] == "Total Sexo") &
    (df_ivs["label_sit_dom"] == "Total Situação de Domicílio")
].copy()

# (IBGE6 já é 6 dígitos, mas vamos garantir)
df_ivs["IBGE6"] = df_ivs["IBGE6"].astype(str).str.zfill(6)

# tentar padronizar nome de município (se existir) para usar depois
nome_col_ivs = pick(df_ivs, ["nome_municipio_uf", "nome_mun_uf", "nome_municipio"])
if nome_col_ivs is not None and nome_col_ivs != "nome_municipio_uf":
    df_ivs = df_ivs.rename(columns={nome_col_ivs: "nome_municipio_uf"})

# =========================================================
# 5) df_pop  (população/área MG 2022)
# =========================================================
# excluir 3 primeiras linhas e a última:
df_pop = pd.read_excel(
    path_pop,
    sheet_name="Tabela",
    header=3,          # usa a 4ª linha como cabeçalho (0-based)
    skipfooter=1,      # exclui última linha (total)
    engine="openpyxl"
)

# renomear primeira coluna para nome_municipio_uf
primeira_col = df_pop.columns[0]
df_pop = df_pop.rename(columns={primeira_col: "nome_municipio_uf"})

# renomear colunas de interesse
rename_pop = {
    "População residente (Pessoas)": "pop_2022",
    "Área da unidade territorial (Quilômetros quadrados)": "area_km2_2022",
    "Densidade demográfica (Habitante por quilômetro quadrado)": "densidade_demografica_pop_km2_2022"
}
df_pop = df_pop.rename(columns=rename_pop)

# manter só colunas relevantes (e o nome, claro)
colunas_pop = ["nome_municipio_uf", "pop_2022", "area_km2_2022", "densidade_demografica_pop_km2_2022"]
for c in colunas_pop:
    if c not in df_pop.columns:
        df_pop[c] = np.nan
df_pop = df_pop[colunas_pop].copy()

# =========================================================
# 6) df_municipio = df_obitos ⨝ df_ivs (por IBGE6)
# =========================================================
df_municipio = df_obitos.merge(df_ivs, on="IBGE6", how="left")

# =========================================================
# 7) merge df_municipio com df_pop (por nome_municipio_uf)
# =========================================================
# garantir que ambos têm nome_municipio_uf
if "nome_municipio_uf" not in df_municipio.columns:
    # tenta criar a partir de alguma coluna do IVS, se não existir
    nome_col_ivs2 = pick(df_municipio, ["nome_municipio_uf", "nome_mun_uf", "nome_municipio", "municipio"])
    if nome_col_ivs2 is not None and nome_col_ivs2 != "nome_municipio_uf":
        df_municipio = df_municipio.rename(columns={nome_col_ivs2: "nome_municipio_uf"})

df_municipio_pop = df_municipio.merge(df_pop, on="nome_municipio_uf", how="left")

# =========================================================
# 8) SALVAR RESULTADO
# =========================================================
with pd.ExcelWriter(path_saida, engine="openpyxl") as writer:
    df_municipio_pop.to_excel(writer, sheet_name="caract_municipio_mg", index=False)

print("Arquivo gerado:", path_saida.resolve())
df_municipio_pop.head()


Arquivo gerado: C:\Users\renatarta\OneDrive - Presidência da República\Documentos\MBA_ENAP\Projeto TCC\TCC\caract_municipio_mg.xlsx


,IBGE6,obitos_total,obitos_icsap,id,nivel,ano,regiao,nome_regiao,outros_territs,uf,...,t_empregador_18m,t_formal_18m,t_fundc_ocup18m,t_medioc_ocup18m,t_supec_ocup18m,t_renda_todos_trabalhos,t_nremunerado_18m,pop_2022,area_km2_2022,densidade_demografica_pop_km2_2022
0,310010,66,17,56123,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,...,"0,20","41,97","42,63","20,54","6,87","920,10","9,80",6272,880.461,7.12
1,310020,223,62,56125,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,...,"1,88","55,37","39,89","25,32","7,97","930,85","4,49",22675,1817.067,12.48
2,310030,125,46,56127,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,...,"1,22","35,42","38,01","24,96","8,41","685,72","15,39",13927,470.551,29.60
3,310040,30,9,56129,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,...,"0,00","47,42","43,76","28,46","8,86","646,71","7,65",3909,101.886,38.37
4,310050,71,31,56219,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,...,"1,14","39,24","34,69","21,89","5,28","514,60","18,93",8943,815.422,10.97


In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

# ============================
# CONFIGURAÇÃO DE CAMINHOS
# ============================
caract_path = Path("caract_municipio_mg.xlsx")
ubs_path    = Path("ubs_mg.xlsx")
saida_path  = Path("caract_municipio_mg_final.xlsx")

# ============================
# 1) LER caract_municipio_mg
# ============================
df = pd.read_excel(caract_path, dtype=str)

# garantir IBGE6 como string de 6 dígitos
if "IBGE6" in df.columns:
    df["IBGE6"] = df["IBGE6"].astype(str).str.extract(r"(\d+)")[0].str.zfill(6)

# renomear colunas de óbitos
rename_map = {}
if "obitos_total" in df.columns:
    rename_map["obitos_total"] = "obitos_total_2022"
if "obitos_icsap" in df.columns:
    rename_map["obitos_icsap"] = "obitos_icsap_2022"
df = df.rename(columns=rename_map)


# ============================
# 2) LER UBS E AGRUPAR POR MUNICÍPIO
# ============================
ubs = pd.read_excel(ubs_path, dtype=str)

# garantir CO_MUNICIPIO_GESTOR com 6 dígitos
ubs["CO_MUNICIPIO_GESTOR"] = (
    ubs["CO_MUNICIPIO_GESTOR"]
    .astype(str)
    .str.extract(r"(\d+)")[0]
    .str.zfill(6)
)

# groupby para obter quantidade de UBS por município
ubs_group = (
    ubs.groupby("CO_MUNICIPIO_GESTOR")
       .size()
       .reset_index(name="qtd_ubs_mun_2022")
)

# ============================
# 3) MERGE caract_municipio_mg + ubs_group
# ============================
df = df.merge(
    ubs_group,
    left_on="IBGE6",
    right_on="CO_MUNICIPIO_GESTOR",
    how="left"
)

# excluir CO_MUNICIPIO_GESTOR após o merge
if "CO_MUNICIPIO_GESTOR" in df.columns:
    df = df.drop(columns=["CO_MUNICIPIO_GESTOR"])

# excluir qtd_ubs_mun_2022_y após o merge
if "qtd_ubs_mun_2022_y" in df.columns:
    df = df.drop(columns=["qtd_ubs_mun_2022_y"])

# renomear colunas de qtd_ubs_mun_2022_x
rename_map = {}
if "qtd_ubs_mun_2022_x" in df.columns:
    rename_map["qtd_ubs_mun_2022_x"] = "qtd_ubs_mun_2022"
df = df.rename(columns=rename_map)

# ============================
# 4) LISTAS DE COLUNAS NUMÉRICAS
# ============================

# floats com 3 casas
float3_cols = [
    "ivs","ivs_infraestrutura_urbana","ivs_capital_humano","ivs_renda_e_trabalho",
    "idhm","idhm_long","idhm_educ","idhm_renda","idhm_educ_sub_esc","idhm_educ_sub_freq",
    "t_sem_agua_esgoto","t_sem_lixo","t_vulner_mais1h","t_mort1","t_c0a5_fora",
    "t_c6a14_fora","t_m10a17_filho","t_mchefe_fundin_fmenor","t_analf_15m",
    "t_cdom_fundin","t_p15a24_nada","t_vulner","t_desocup18m","t_p18m_fundin_informal",
    "t_vulner_depende_idosos","t_atividade10a14","espvida","t_pop18m_fundc",
    "t_pop5a6_escola","t_pop11a13_ffun","t_pop15a17_fundc","t_pop18a20_medioc",
    "renda_per_capita","t_fmor5","t_razdep","t_fectot","t_env",
    "t_eletrica","t_densidadem2","t_analf_18m","t_analf_25m","rdpc_def_vulner",
    "t_renda_trab","i_gini","t_carteira_18m","t_scarteira_18m","t_setorpublico_18m",
    "t_contapropria_18m","t_empregador_18m","t_formal_18m","t_fundc_ocup18m",
    "t_medioc_ocup18m","t_supec_ocup18m","t_renda_todos_trabalhos",
    "t_nremunerado_18m",
    "area_km2_2022","densidade_demografica_pop_km2_2022",
]

# inteiras
int_cols = [
    "populacao","vulner15a24","mchefe_fmenor","vulner_dia","dom_vulner_idoso",
    "pop0a1","pop1a3","pop4","pop5","pop6","pop6a10","pop6a17","pop11a13","pop11a14",
    "pop12a14","pop15m","pop15a17","pop15a24","pop16a18","pop18m","pop18a20",
    "pop18a24","pop19a21","pop25m","pop65m","pea10m","pea10a14","pea15a17","pea18m",
    "pop_2022",
    "obitos_total_2022","obitos_icsap_2022","qtd_ubs_mun_2022",
]

# colunas numéricas que realmente existem
float3_exist = [c for c in float3_cols if c in df.columns]
int_exist    = [c for c in int_cols    if c in df.columns]
num_cols_exist = float3_exist + int_exist

# ============================
# 5) LIMPEZA NUMÉRICA
# ============================

for c in num_cols_exist:
    # pega a coluna como string
    col = df[c].astype(str).str.strip()

    # troca "X" por "0"
    col = col.replace("X", "0")

    col = col.str.replace(",", ".", regex=False)

    # converte para numérico
    col = pd.to_numeric(col, errors="coerce").fillna(0)

    df[c] = col

# aplicar tipos finais
for c in float3_exist:
    df[c] = df[c].astype(float).round(3)

for c in int_exist:
    df[c] = df[c].astype(int)

# ============================
# 6) REORDENAR COLUNAS
# ============================

col_order = [
    "IBGE6",
    "id",
    "nivel",
    "ano",
    "regiao",
    "nome_regiao",
    "outros_territs",
    "uf",
    "nome_uf",
    "rm",
    "nome_rm",
    "municipio",
    "nome_municipio_uf",
    "udh",
    "nome_udh",
    "label_cor",
    "label_sexo",
    "label_sit_dom",
    "ivs",
    "ivs_infraestrutura_urbana",
    "ivs_capital_humano",
    "ivs_renda_e_trabalho",
    "idhm",
    "idhm_long",
    "idhm_educ",
    "idhm_renda",
    "idhm_educ_sub_esc",
    "idhm_educ_sub_freq",
    "prosp_soc",
    "t_sem_agua_esgoto",
    "t_sem_lixo",
    "t_vulner_mais1h",
    "t_mort1",
    "t_c0a5_fora",
    "t_c6a14_fora",
    "t_m10a17_filho",
    "t_mchefe_fundin_fmenor",
    "t_analf_15m",
    "t_cdom_fundin",
    "t_p15a24_nada",
    "t_vulner",
    "t_desocup18m",
    "t_p18m_fundin_informal",
    "t_vulner_depende_idosos",
    "t_atividade10a14",
    "espvida",
    "t_pop18m_fundc",
    "t_pop5a6_escola",
    "t_pop11a13_ffun",
    "t_pop15a17_fundc",
    "t_pop18a20_medioc",
    "renda_per_capita",
    "populacao",
    "t_fmor5",
    "t_razdep",
    "t_fectot",
    "t_env",
    "vulner15a24",
    "mchefe_fmenor",
    "vulner_dia",
    "dom_vulner_idoso",
    "pop0a1",
    "pop1a3",
    "pop4",
    "pop5",
    "pop6",
    "pop6a10",
    "pop6a17",
    "pop11a13",
    "pop11a14",
    "pop12a14",
    "pop15m",
    "pop15a17",
    "pop15a24",
    "pop16a18",
    "pop18m",
    "pop18a20",
    "pop18a24",
    "pop19a21",
    "pop25m",
    "pop65m",
    "pea10m",
    "pea10a14",
    "pea15a17",
    "pea18m",
    "t_eletrica",
    "t_densidadem2",
    "t_analf_18m",
    "t_analf_25m",
    "rdpc_def_vulner",
    "t_renda_trab",
    "i_gini",
    "t_carteira_18m",
    "t_scarteira_18m",
    "t_setorpublico_18m",
    "t_contapropria_18m",
    "t_empregador_18m",
    "t_formal_18m",
    "t_fundc_ocup18m",
    "t_medioc_ocup18m",
    "t_supec_ocup18m",
    "t_renda_todos_trabalhos",
    "t_nremunerado_18m",
    "pop_2022",
    "area_km2_2022",
    "densidade_demografica_pop_km2_2022",
    "obitos_total_2022",
    "obitos_icsap_2022",
    "qtd_ubs_mun",
]

cols_final = [c for c in col_order if c in df.columns] + \
             [c for c in df.columns if c not in col_order]

df = df[cols_final]

# ============================
# 7) EXPORTAR
# ============================
with pd.ExcelWriter(saida_path, engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="caract_municipio_mg", index=False)

print("Arquivo gerado:", saida_path.resolve())
df.head()

Arquivo gerado: C:\Users\renatarta\OneDrive - Presidência da República\Documentos\MBA_ENAP\Projeto TCC\TCC\caract_municipio_mg_final.xlsx


,IBGE6,id,nivel,ano,regiao,nome_regiao,outros_territs,uf,nome_uf,rm,...,t_medioc_ocup18m,t_supec_ocup18m,t_renda_todos_trabalhos,t_nremunerado_18m,pop_2022,area_km2_2022,densidade_demografica_pop_km2_2022,obitos_total_2022,obitos_icsap_2022,qtd_ubs_mun_2022
0,310010,56123,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,Minas Gerais,-,...,20.54,6.87,920.10,9.80,6272,880.461,7.12,66,17,2
1,310020,56125,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,Minas Gerais,-,...,25.32,7.97,930.85,4.49,22675,1817.067,12.48,223,62,6
2,310030,56127,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,Minas Gerais,-,...,24.96,8.41,685.72,15.39,13927,470.551,29.60,125,46,7
3,310040,56129,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,Minas Gerais,-,...,28.46,8.86,646.71,7.65,3909,101.886,38.37,30,9,3
4,310050,56219,"regiao,uf,rm,municipio",2010,3,SUDESTE,-,31,Minas Gerais,-,...,21.89,5.28,514.60,18.93,8943,815.422,10.97,71,31,6


# Separação das variáveis para o projeto de ML

In [8]:
import pandas as pd
df = pd.read_excel("caract_municipio_mg_final.xlsx")

rename_map = {
    "municipio": "IBGE7",
    "populacao": "pop_2010",
}

df = df.rename(columns=rename_map)

map_prosp = {
    "Muito Alto": 1,
    "Alto": 2,
    "Médio": 3,   # com acento
    "Medio": 3,   # sem acento, por segurança
    "Baixo": 4,
    "Muito Baixo": 5,
}

df["prosp_soc"] = df["prosp_soc"].map(map_prosp)

In [9]:
cols_final = [
    "IBGE7",               # 12 - municipio renomeado
    "ivs",                         # 19  - float(3)
    "idhm",                        # 23  - float(3)
    "prosp_soc",                   # 29  - numérico (1–5)
    "espvida",                     # 46  - float(2)
    "renda_per_capita",            # 52  - float(2)
    "i_gini",                      # 92  - float(3)
    "pop_2022",                              # 104 - int
    "densidade_demografica_pop_km2_2022",    # 106 - float
    "obitos_total_2022",                     # 107 - int
    "obitos_icsap_2022",                     # 108 - int
    "qtd_ubs_mun_2022",                      # 109 - int
]

df_final = df[cols_final].copy()

In [10]:
float_3 = [
    "ivs",
    "idhm",
    "i_gini",
]

df_final[float_3] = df_final[float_3].astype(float).round(3)

float_2 = [
    "espvida",
    "renda_per_capita",
    ]

df_final[float_2] = df_final[float_2].astype(float).round(2)

int_cols = [
    "pop_2022",
    "obitos_total_2022",
    "obitos_icsap_2022",
    "qtd_ubs_mun_2022",
]

df_final[int_cols] = df_final[int_cols].astype("Int64")  # suporta NaN

str_cols = [
    "IBGE7",
]

df_final[str_cols] = df_final[str_cols].astype("string")

In [11]:
from pathlib import Path
saida_path  = Path("dataset_mun_MG.xlsx")
with pd.ExcelWriter(saida_path, engine="openpyxl") as writer:
    df_final.to_excel(writer, sheet_name="caract_municipio_mg", index=False)

print("Arquivo gerado:", saida_path.resolve())
df_final.head()

Arquivo gerado: C:\Users\renatarta\OneDrive - Presidência da República\Documentos\MBA_ENAP\Projeto TCC\TCC\dataset_mun_MG.xlsx


,IBGE7,ivs,idhm,prosp_soc,espvida,renda_per_capita,i_gini,pop_2022,densidade_demografica_pop_km2_2022,obitos_total_2022,obitos_icsap_2022,qtd_ubs_mun_2022
0,3100104,0.232,0.689,2,75.35,596.18,0.47,6272,7.12,66,17,2
1,3100203,0.290,0.698,2,75.90,707.24,0.54,22675,12.48,223,62,6
2,3100302,0.282,0.654,2,74.36,444.32,0.52,13927,29.60,125,46,7
3,3100401,0.335,0.630,3,74.36,357.03,0.48,3909,38.37,30,9,3
4,3100500,0.571,0.610,4,75.66,325.42,0.48,8943,10.97,71,31,6
